## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [2]:
df = pd.read_csv('redfin_2023-08-02-06-00-26.csv')

## Data Clean

In [3]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [4]:
df[['SOLD DATE','PRICE','ADDRESS']].value_counts()

SOLD DATE      PRICE      ADDRESS                  
August-1-2023  90000.0    651 Okeechobee Blvd #907     1
May-11-2023    559000.0   600 S Dixie Hwy #341         1
               992500.0   2800 S Ocean Blvd Unit 4b    1
               3025000.0  315 S Lake Dr Unit 2a        1
May-12-2023    56400.0    244 Windsor K                1
                                                      ..
June-15-2023   110000.0   2520 Dudley Dr Unit i        1
               115000.0   33 Stratford Ln Unit B       1
                          55 Suffolk B                 1
               119000.0   142 Brighton D               1
May-9-2023     950000.0   3912 S Ocean Blvd #1103      1
Name: count, Length: 2384, dtype: int64

In [5]:
df = df.dropna(subset=['SOLD DATE','PRICE','ADDRESS'])

In [6]:
# Define list of desired months (excluding current month)
desired_months = ['July']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [7]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    616
Name: count, dtype: int64
-------
$/SQUARE FEET
False    614
True       2
Name: count, dtype: int64
-------
YEAR BUILT
False    616
Name: count, dtype: int64
-------


In [8]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [9]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [10]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
563,PAST SALE,July-24-2023,Condo/Co-op,5133 Brisata Cir Unit T,Boynton Beach,FL,33437.0,270.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/5133-B...,Beaches MLS,RX-10865542,N,Y,26.530645,-80.126895
606,PAST SALE,July-18-2023,Condo/Co-op,10 Stratford Dr Unit B,Boynton Beach,FL,33436.0,60000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/10-Str...,Beaches MLS,RX-10839018,N,Y,26.485038,-80.105626
193,PAST SALE,July-27-2023,Condo/Co-op,1842 Bridgewood Dr,Boca Raton,FL,33434.0,65000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1842-Brid...,Beaches MLS,RX-10887802,N,Y,26.373894,-80.164420
571,PAST SALE,July-21-2023,Condo/Co-op,103 Hastings G,West Palm Beach,FL,33417.0,70000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/103-...,Beaches MLS,RX-10860837,N,Y,26.722612,-80.125769
464,PAST SALE,July-17-2023,Condo/Co-op,111 E Tiffany Dr #2,Mangonia Park,FL,33407.0,75000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Mangonia-Park/111-E-...,Beaches MLS,RX-10874553,N,Y,26.756361,-80.083131
142,PAST SALE,July-10-2023,Condo/Co-op,15 Windsor A,West Palm Beach,FL,33417.0,76000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/15-W...,Beaches MLS,RX-10887963,N,Y,26.709731,-80.136068
528,PAST SALE,July-7-2023,Condo/Co-op,114 Cambridge #114,West Palm Beach,FL,33417.0,76000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/114-...,Beaches MLS,F10368617,N,Y,26.720353,-80.130764
285,PAST SALE,July-14-2023,Condo/Co-op,74 Camden C,West Palm Beach,FL,33417.0,76000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/74-C...,Beaches MLS,RX-10888935,N,Y,26.714762,-80.136017
356,PAST SALE,July-12-2023,Condo/Co-op,135 Berkshire F,West Palm Beach,FL,33417.0,78000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/135-...,Beaches MLS,RX-10886405,N,Y,26.713849,-80.132421
138,PAST SALE,July-28-2023,Condo/Co-op,137 Canterbury F #137,West Palm Beach,FL,33417.0,79000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/137-...,Beaches MLS,RX-10900434,N,Y,26.720227,-80.132277


In [11]:
print(df_filtered['URL'].iloc[193])

https://www.redfin.com/FL/Boca-Raton/1842-Bridgewood-Dr-33434/home/42249452


In [12]:
# Correct the prices, if needed
df_filtered.at[563,'PRICE']=(270000)

In [13]:
# # Find problem psf by searching for a '0' value
# df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

In [14]:
# # Corrections, if needed
df_filtered.at[563,'$/SQUARE FEET']=(270000/1374)

In [15]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
606,60000.0,10 Stratford Dr Unit B,Boynton Beach,39.0
193,65000.0,1842 Bridgewood Dr,Boca Raton,52.0
464,75000.0,111 E Tiffany Dr #2,Mangonia Park,77.0
425,80000.0,2687 N Garden Dr #205,Lake Worth,89.0
387,98000.0,4700 Lucerne Lakes Blvd #305,Lake Worth,109.0
615,110000.0,2615 NE 3rd Ct #412,Boynton Beach,112.0
367,153000.0,7903 Willow Spring Dr #1114,Lake Worth,113.0
289,115000.0,130 NE 26th Ave #401,Boynton Beach,117.0
473,115000.0,300 NE 26th #110,Boynton Beach,117.0
285,76000.0,74 Camden C,West Palm Beach,118.0


In [16]:
print(df_filtered.URL.iloc[606])

https://www.redfin.com/FL/Boynton-Beach/10-Stratford-Dr-E-33436/unit-B/home/42333370


In [17]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [18]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [19]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [20]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [21]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [22]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [23]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"July 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [24]:
m.save('index.html')

## Summary Info

In [25]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [26]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [27]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [36]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['URL'])

https://www.redfin.com/FL/West-Palm-Beach/200-Arkona-Ct-33401/unit-1601/home/176986474


In [29]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
Alina Boca Raton Condo, 200 SE Mizner Blvd #417 Boca Raton | Price $6,000,000 | $1,713 psf | Year built: 2021
Least Expensive
Stratford AT Hunters Run Condo, 10 Stratford Dr Unit B Boynton Beach | Price $60,000 | $39 psf | Year built: 1980
Highest Price Per Square Foot
Island Villas, 225 Everglade Ave #6 Palm Beach | Price $2,850,000 | $2,593 psf | Year built: 1985
Lowest Price Per Square Foot
Stratford AT Hunters Run Condo, 10 Stratford Dr Unit B Boynton Beach | Price $60,000 | $39 psf | Year built: 1980
Newest
La Clara, 200 Arkona Ct #1601 West Palm Beach | Price $4,651,480 | $1,473 psf | Year built: 2023
Oldest
Peruvian Court Condo, 173 Peruvian Ave #5 Palm Beach | Price $1,750,000 | $1,632 psf | Year built: 1923


## Time on Market Calculator

In [30]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [31]:
from datetime import datetime, timedelta

date1 = datetime(2023, 1, 18) ## List (Earlier) date
date2 = datetime(2023, 7, 18) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

181


## Map URL Snagger

In [32]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [33]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PBC_condo_sales_month_ending_july_2023


## Get Summary Data

In [34]:
df.sort_values(by='PRICE',ascending=False)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
15,PAST SALE,May-19-2023,Condo/Co-op,1625 S Ocean Blvd Unit B-1 North 2/8,Delray Beach,FL,33483.0,10898999.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1625-S-...,Beaches MLS,RX-10848061,N,Y,26.440918,-80.061707
2550,PAST SALE,May-25-2023,Condo/Co-op,1500 Beach Rd #501,Tequesta,FL,33469.0,9410000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/1500-S-Beach...,Beaches MLS,RX-10848698,N,Y,26.961813,-80.079386
707,PAST SALE,June-23-2023,Condo/Co-op,4001 N Ocean Blvd #504,Gulf Stream,FL,33483.0,8000000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Gulf-Stream/4001-N-O...,Beaches MLS,RX-10891626,N,Y,26.501732,-80.052369
318,PAST SALE,May-31-2023,Condo/Co-op,4001 N Ocean Blvd #303,Gulf Stream,FL,33483.0,7550000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Gulf-Stream/4001-N-O...,Beaches MLS,RX-10887460,N,Y,26.501732,-80.052369
111,PAST SALE,May-19-2023,Condo/Co-op,1625 S Ocean Blvd Unit A3-South 13,Delray Beach,FL,33483.0,7499000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1625-S-...,Beaches MLS,RX-10846805,N,Y,26.440922,-80.061722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2574,PAST SALE,May-31-2023,Condo/Co-op,167 Yacht Club Way #206,Hypoluxo,FL,33462.0,400.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hypoluxo/167-Yacht-C...,MARMLS,A11306531,N,Y,26.565431,-80.050436
312,PAST SALE,June-7-2023,Condo/Co-op,7786 Lakeside Blvd #676,Boca Raton,FL,33434.0,310.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/7786-Lake...,Beaches MLS,RX-10885824,N,Y,26.378671,-80.167671
2515,PAST SALE,July-24-2023,Condo/Co-op,5133 Brisata Cir Unit T,Boynton Beach,FL,33437.0,270.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/5133-B...,Beaches MLS,RX-10865542,N,Y,26.530645,-80.126895
355,PAST SALE,May-22-2023,Condo/Co-op,1721 Village Blvd #305,West Palm Beach,FL,33409.0,220.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1721...,Beaches MLS,RX-10878498,N,Y,26.727223,-80.101542


In [35]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 616
--------
Total sale price: $289,055,554
--------
Median sale price: $265,000
--------
Max sale price: $6,000,000
--------
Min sale price: $60,000
------------------------------------------------
PSF INFO
Max price per square foot: $2,593
--------
Min price per square foot: $39
--------
Median price per square foot: $243
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1923.0
----------
Average building age: 1983.810064935065
